In [1]:
import numpy
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
import cv2
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import xml.etree.ElementTree as ET
seed = 7
numpy.random.seed(seed)
size = (100, 100)
image_dir = r'C:\Users\kshruth\Documents\automated parking\implemetation\JPEGImages1'
label_dir = r'C:\Users\kshruth\Documents\automated parking\implemetation\Annotationsnumber'
files = os.listdir(image_dir)
annot = os.listdir(label_dir)

In [4]:
images = []
annotations = []
actual = []
boxes = []
for filename in files:
    path = os.path.join(image_dir,filename)
    img = cv2.imread(path)
    print(filename)
    img = cv2.resize(img,size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    images.append(img)
    height, width = img.shape
    y_scale = 100/height
    x_scale = 100/width
    annot_name = os.path.splitext(filename)[0]+'.xml'
    annot_path = os.path.join(label_dir,annot_name)
    root = ET.parse(annot_path).getroot()
    objects = root.findall('object')
    annotations.append(list(set(objects)))
    box = objects[0].findall('bndbox')[0]
    print(box.find('ymin').text)
    x1,y1,x2,y2 = (int(box.find('xmin').text),
                  int(box.find('ymin').text),
                  int(box.find('xmax').text),
                  int(box.find('ymax').text))
    boxes.append([x_scale*x1,y_scale*y1,x_scale*x2,y_scale*y2])
   
        

2009_006001.jpg
423
2009_006002.jpg
542
2009_006003.jpg
523
2009_006004.jpg
506
2009_006005.jpg
744
2009_006006.jpg
618
2009_006007.jpg
715
2009_006008.jpg
698
2009_006009.jpg
546
2009_006010.jpg
695
2009_006011.jpg
646
2009_006012.jpg
580
2009_006013.jpg
506
2009_006014.jpg
506
2009_006015.jpg
506
2009_006016.jpg
506
2009_006017.jpg
506
2009_006018.jpg
744
2009_006019.jpg
744
2009_006020.jpg
744
2009_006021.jpg
744
2009_006022.jpg
744
2009_006023.jpg
744
2009_006024.jpg
618
2009_006025.jpg
618
2009_006026.jpg
618
2009_006027.jpg
618
2009_006028.jpg
618
2009_006029.jpg
618
2009_006030.jpg
715
2009_006031.jpg
715
2009_006032.jpg
715
2009_006033.jpg
715
2009_006034.jpg
715
2009_006035.jpg
715
2009_006036.jpg
698
2009_006037.jpg
698
2009_006038.jpg
698
2009_006039.jpg
698
2009_006040.jpg
698
2009_006041.jpg
698
2009_006042.jpg
546
2009_006043.jpg
546
2009_006044.jpg
546
2009_006045.jpg
546
2009_006046.jpg
546
2009_006047.jpg
546
2009_006048.jpg
695
2009_006049.jpg
695
2009_006050.jpg
695


In [5]:
def cross_validate(xs,ys):
    X_train, X_test, y_train, y_test = train_test_split(xs, ys, test_size = 0, random_state = 0)
    return X_train,X_test, y_train, y_test
print("boxes")
images = numpy.array(images)
boxes = numpy.array(boxes)

X_train, X_test, y_train, y_test = cross_validate(images, boxes)
print(len(images))
print(len(boxes))
print(X_train[0])
test = []
test.append(X_train[0])

boxes
78
78
[[216 213 214 ... 154 154 154]
 [222 218 208 ... 155 155 156]
 [221 217 218 ... 157 157 157]
 ...
 [123 121 135 ... 118  84  92]
 [123 136 128 ...  99 100  99]
 [133 125 128 ...  86  79 110]]


In [6]:
X_train=numpy.array(X_train).astype('float32')
X_test =numpy.array(X_test).astype('float32')
test =numpy.array(test).astype('float32')
X_train = X_train/255.0
X_test = X_test/255.0
test = test/255.0 
X_train_mean = numpy.mean(X_train, axis = 0)
print(X_train.shape[1:])
nRows, nCols = X_train.shape[1:]

(100, 100)


In [7]:
from keras import backend as K
K.set_image_dim_ordering('tf')
nDims = 1
X_train = X_train.reshape(X_train.shape[0], nRows, nCols, nDims)
X_test = X_test.reshape(X_test.shape[0], nRows, nCols, nDims)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(100,100,1)))
#model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100,100,1)))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
#model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(100,100,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='relu'))


#sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)


model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


history = model.fit(X_train, y_train, batch_size=16, epochs=6,verbose=1, validation_split=0.1)

m = model.evaluate(X_test, y_test)

Train on 70 samples, validate on 8 samples
Epoch 1/6
70/70 [==============================] - 6s 87ms/step - loss: 515436.7875 - acc: 0.0429 - val_loss: 336483.5312 - val_acc: 0.0000e+00
Epoch 2/6
70/70 [==============================] - 5s 73ms/step - loss: 197669.4842 - acc: 0.2571 - val_loss: 35640.5586 - val_acc: 0.6250
Epoch 3/6
70/70 [==============================] - 5s 74ms/step - loss: 57834.2598 - acc: 0.4714 - val_loss: 68932.6250 - val_acc: 0.3750
Epoch 4/6
70/70 [==============================] - 5s 74ms/step - loss: 60242.5787 - acc: 0.4000 - val_loss: 29648.1172 - val_acc: 0.3750
Epoch 5/6
70/70 [==============================] - 5s 78ms/step - loss: 48626.8540 - acc: 0.3143 - val_loss: 46284.9453 - val_acc: 0.6250
Epoch 6/6
70/70 [==============================] - 5s 76ms/step - loss: 44787.1086 - acc: 0.4714 - val_loss: 22881.6602 - val_acc: 0.6250


In [33]:
testimage = []
test ="C:/Users/kshruth/Pictures/bike.png" 
img = cv2.imread(test)
img = cv2.resize(img,size)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
testimage.append(img)
height, width = img.shape
y_scale = 100/height
x_scale = 100/width
testimage = numpy.array(testimage)
X_test =numpy.array(testimage).astype('float32')
X_test = X_test/255.0
nDims = 1
X_test = X_test.reshape(X_test.shape[0], nRows, nCols, nDims)
predit = model.predict(X_test)
print(predit)

[[595.7471  478.73468 609.254   418.82492]]


In [34]:
x = "C:/Users/kshruth/Documents/automated parking/implemetation/JPEGImages/2009_006004.jpg"
print(x)
i=0
im = cv2.imread(test,cv2.IMREAD_COLOR)
cv2.rectangle(im,(predit[i][0],predit[i][1]),(predit[i][2],predit[i][3]),(0,0,255),5)
cv2.imshow('image',im)
cv2.waitKey()
cv2.destroyAllWindows()
p = [[800,1000,300,600]]

C:/Users/kshruth/Documents/automated parking/implemetation/JPEGImages/2009_006004.jpg
